In [4]:
# ANÁLISIS EXPLORATORIO DE DATOS - VERSIÓN OPTIMIZADA
# =====================================================

import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# 1. CONFIGURACIÓN Y FUNCIONES AUXILIARES
# =============================================================================

def leer_archivos_defunciones(ruta_carpeta, año_inicio, año_fin):
    """Lee y unifica archivos de defunciones en un rango de años"""
    ruta = Path(ruta_carpeta)
    archivos = [arch for arch in ruta.glob("Defun*.txt") 
                if año_inicio <= int(arch.stem[-4:]) <= año_fin]
    
    dfs = []
    for archivo in tqdm(archivos, desc="Leyendo archivos"):
        try:
            df = pd.read_csv(archivo, sep='\t', encoding='utf-8', 
                           low_memory=False, dtype=str)
            df.columns = df.columns.str.upper()
            dfs.append(df)
        except Exception as e:
            print(f"Error en {archivo.name}: {e}")
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def crear_mapeo_descripciones():
    """Crea diccionarios de mapeo para las descripciones"""
    return {
        'A_DEFUN': {
            '1': 'CABECERA MUNICIPAL',
            '2': 'RESTO',
            '3': 'SIN INFORMACIÓN'
        },
        'SEXO': {
            '1': 'MASCULINO',
            '2': 'FEMENINO'
        },
        'EST_CIVIL': {
            '1': 'SOLTERO',
            '2': 'CASADO',
            '3': 'VIUDO',
            '4': 'EN UNIÓN LIBRE, DIVORCIADO Y OTRO',
            '5': 'SIN INFORMACIÓN'
        },
        'SIT_DEFUN': {
            '1': 'HOSPITAL O CLINICA',
            '2': 'CASA',
            '3': 'OTRO SITIO',
            '4': 'SIN INFORMACIÓN'
        },
        'GRU_ED1': {
            '01': 'MENORES DE UN DÍA', '02': 'DE 1 A 6 DÍAS',
            '03': 'DE 7 A 29 DÍAS', '04': 'DE 1 A 5 MESES',
            '05': 'DE 6 A 11 MESES', '06': 'DE UN AÑO',
            '07': 'DE 2 A 4 AÑOS', '08': 'DE 5 A 9 AÑOS',
            '09': 'DE 10 A 14 AÑOS', '10': 'DE 15 A 19 AÑOS',
            '11': 'DE 20 A 24 AÑOS', '12': 'DE 25 A 29 AÑOS',
            '13': 'DE 30 A 34 AÑOS', '14': 'DE 35 A 39 AÑOS',
            '15': 'DE 40 A 44 AÑOS', '16': 'DE 45 A 49 AÑOS',
            '17': 'DE 50 A 54 AÑOS', '18': 'DE 55 A 59 AÑOS',
            '19': 'DE 60 A 64 AÑOS', '20': 'DE 65 A 69 AÑOS',
            '21': 'DE 70 A 74 AÑOS', '22': 'DE 75 A 79 AÑOS',
            '23': 'DE 80 A 84 AÑOS', '24': 'DE 85 Y MÁS AÑOS',
            '25': 'EDAD DESCONOCIDA'
        }
    }

def homologar_departamentos_residencia(df, divipola):
    """Homologa departamentos y municipios de residencia"""
    # Preparar códigos
    df['CODPTORE'] = df['CODPTORE'].astype(str).str.zfill(2)
    df['CODMUNRE'] = df['CODMUNRE'].astype(str).str.zfill(3)
    
    # Corregir código antiguo 83 -> 18 (Caquetá)
    df.loc[df['CODPTORE'] == '83', 'CODPTORE'] = '18'
    
    # Asegurar que no hay duplicados en divipola
    divipola_unico = divipola.drop_duplicates(subset=['COD_DPTO', 'COD_MUNIC'])
    
    # Merge principal
    df = df.merge(
        divipola_unico[['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']],
        left_on=['CODPTORE', 'CODMUNRE'],
        right_on=['COD_DPTO', 'COD_MUNIC'],
        how='left',
        suffixes=('', '_RESIDENCIA')
    )
    
    print(f"   Registros después del merge: {len(df):,}")
    
    # Completar nulos con cabeceras municipales
    cabeceras = divipola_unico[divipola_unico['COD_MUNIC'] == '001'][
        ['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']
    ].drop_duplicates(subset=['COD_DPTO']).rename(columns={
        'NOMBRE_DEPARTAMENTO': 'DEPT_CAB',
        'NOMBRE_MUNICIPIO': 'MUNIC_CAB'
    })
    
    df = df.merge(cabeceras, left_on='CODPTORE', right_on='COD_DPTO', 
                  how='left', suffixes=('', '_y'))
    
    print(f"   Registros después de cabeceras: {len(df):,}")
    
    mask = df['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna()
    df.loc[mask, 'NOMBRE_DEPARTAMENTO_RESIDENCIA'] = df.loc[mask, 'DEPT_CAB']
    df.loc[mask, 'NOMBRE_MUNICIPIO_RESIDENCIA'] = df.loc[mask, 'MUNIC_CAB']
    
    # Marcar los que no se encontraron
    df['NOMBRE_DEPARTAMENTO_RESIDENCIA'].fillna('NO REGISTRA', inplace=True)
    df['NOMBRE_MUNICIPIO_RESIDENCIA'].fillna('NO REGISTRA', inplace=True)
    
    # Limpiar columnas temporales
    cols_drop = ['DEPT_CAB', 'MUNIC_CAB', 'COD_DPTO_y', 'COD_DPTO_RESIDENCIA', 
                 'COD_MUNIC_RESIDENCIA']
    df.drop([c for c in cols_drop if c in df.columns], axis=1, inplace=True)
    
    return df

def buscar_cie9_optimizado(c_bas1_val, cie_dict):
    """Búsqueda optimizada de códigos CIE-9 usando diccionario"""
    if pd.isna(c_bas1_val):
        return None
    
    c_bas1_str = str(c_bas1_val).strip()
    
    # Búsqueda directa
    if c_bas1_str in cie_dict:
        return cie_dict[c_bas1_str]
    
    # Solo para códigos numéricos
    if not c_bas1_str.isdigit():
        return None
    
    # Variaciones con prefijos (E, V, M, N)
    for prefijo in ['E', 'V', 'M', 'N']:
        # Con 4 dígitos
        if len(c_bas1_str) == 4:
            codigo = f"{prefijo}{c_bas1_str}"
            if codigo in cie_dict:
                return cie_dict[codigo]
        
        # Con 3 primeros dígitos
        if len(c_bas1_str) >= 3:
            codigo = f"{prefijo}{c_bas1_str[:3]}"
            if codigo in cie_dict:
                return cie_dict[codigo]
        
        # Sin ceros a la izquierda
        sin_ceros = c_bas1_str.lstrip('0')
        if sin_ceros and sin_ceros != c_bas1_str:
            if len(sin_ceros) >= 3:
                codigo = f"{prefijo}{sin_ceros[:3]}"
                if codigo in cie_dict:
                    return cie_dict[codigo]
    
    return None

# =============================================================================
# 2. PROCESO PRINCIPAL
# =============================================================================

print("=" * 60)
print("ANÁLISIS EXPLORATORIO DE DATOS - DEFUNCIONES 1979-1991")
print("=" * 60)

# 2.1. Leer archivos de defunciones
print("\n1. LEYENDO ARCHIVOS DE DEFUNCIONES...")
df_defun = leer_archivos_defunciones("data/raw/Muertes", 1979, 1991)
print(f"   Total registros: {len(df_defun):,}")
print(f"   Columnas: {list(df_defun.columns)}")

# 2.2. Leer DIVIPOLA (códigos de departamentos y municipios)
print("\n2. LEYENDO DIVIPOLA...")
divipola = pd.read_csv("data/raw/Referenciales/DIVIPOLA_CentrosPoblados.csv",
                       encoding='latin-1', sep=';', dtype=str)

print(f"   Columnas disponibles: {list(divipola.columns)}")

# Identificar las columnas correctas (pueden tener nombres diferentes)
# Buscar columnas que contengan las palabras clave
col_cod_mun = [c for c in divipola.columns if 'Municipio' in c and 'digo' in c][0]
col_nom_dpto = [c for c in divipola.columns if 'Departamento' in c and 'Nombre' in c][0]
col_nom_mun = [c for c in divipola.columns if 'Municipio' in c and 'Nombre' in c][0]

print(f"   Usando columnas: {col_cod_mun}, {col_nom_dpto}, {col_nom_mun}")

# Extraer códigos de departamento y municipio del código de 5 dígitos
divipola['COD_DPTO'] = divipola[col_cod_mun].str[:2]
divipola['COD_MUNIC'] = divipola[col_cod_mun].str[2:]

divipola_merge = divipola[['COD_DPTO', 'COD_MUNIC', col_nom_dpto, col_nom_mun]].copy()
divipola_merge.columns = ['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']

# Eliminar duplicados
divipola_merge = divipola_merge.drop_duplicates(subset=['COD_DPTO', 'COD_MUNIC'])

print(f"   Registros únicos: {len(divipola_merge):,}")
print(f"   Muestra:")
print(divipola_merge.head(3)[['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']])

# 2.3. Homologar departamento y municipio de defunción
print("\n3. HOMOLOGANDO UBICACIÓN DE DEFUNCIÓN...")
print(f"   Registros iniciales: {len(df_defun):,}")

df_defun['COD_DPTO'] = df_defun['COD_DPTO'].astype(str).str.zfill(2)
df_defun['COD_MUNIC'] = df_defun['COD_MUNIC'].astype(str).str.zfill(3)
df_defun.loc[df_defun['COD_DPTO'] == '83', 'COD_DPTO'] = '18'

print(f"   Ejemplo códigos en datos: DPTO={df_defun['COD_DPTO'].iloc[0]}, MUN={df_defun['COD_MUNIC'].iloc[0]}")

# Asegurar que divipola_merge no tiene duplicados
divipola_unico = divipola_merge.drop_duplicates(subset=['COD_DPTO', 'COD_MUNIC'])
print(f"   Códigos únicos en DIVIPOLA: {len(divipola_unico):,}")

df_defun = df_defun.merge(divipola_unico, on=['COD_DPTO', 'COD_MUNIC'], how='left')
print(f"   Registros después del merge: {len(df_defun):,}")

# Completar nulos con cabeceras
cabeceras = divipola_unico[divipola_unico['COD_MUNIC'] == '001'][
    ['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']
].drop_duplicates(subset=['COD_DPTO'])

print(f"   Cabeceras municipales: {len(cabeceras):,}")

df_defun = df_defun.merge(cabeceras, on='COD_DPTO', how='left', suffixes=('', '_CAB'))
print(f"   Registros después de cabeceras: {len(df_defun):,}")

mask = df_defun['NOMBRE_DEPARTAMENTO'].isna()
print(f"   Registros con NOMBRE_DEPARTAMENTO nulo: {mask.sum():,}")

df_defun.loc[mask, 'NOMBRE_DEPARTAMENTO'] = df_defun.loc[mask, 'NOMBRE_DEPARTAMENTO_CAB']
df_defun.loc[mask, 'NOMBRE_MUNICIPIO'] = df_defun.loc[mask, 'NOMBRE_MUNICIPIO_CAB']
df_defun.drop(['NOMBRE_DEPARTAMENTO_CAB', 'NOMBRE_MUNICIPIO_CAB'], axis=1, inplace=True)

print(f"   Coincidencias finales: {df_defun['NOMBRE_DEPARTAMENTO'].notna().sum():,}")

# 2.4. Homologar lugar de residencia
print("\n4. HOMOLOGANDO LUGAR DE RESIDENCIA...")
print(f"   Registros antes: {len(df_defun):,}")
df_defun = homologar_departamentos_residencia(df_defun, divipola_merge)
print(f"   Registros después: {len(df_defun):,}")
print(f"   Coincidencias: {(df_defun['NOMBRE_DEPARTAMENTO_RESIDENCIA'] != 'NO REGISTRA').sum():,}")

# 2.5. Crear campos descriptivos
print("\n5. CREANDO CAMPOS DESCRIPTIVOS...")
mapeos = crear_mapeo_descripciones()

df_defun['A_DEFUN_DESC'] = df_defun['A_DEFUN'].map(mapeos['A_DEFUN'])
df_defun['SEXO_DESC'] = df_defun['SEXO'].map(mapeos['SEXO'])
df_defun['EST_CIVIL_DESC'] = df_defun['EST_CIVIL'].map(mapeos['EST_CIVIL'])
df_defun['SIT_DEFUN_DESC'] = df_defun['SIT_DEFUN'].map(mapeos['SIT_DEFUN'])

df_defun['GRU_ED1_STR'] = df_defun['GRU_ED1'].astype(str).str.zfill(2)
df_defun['GRU_ED1_DESC'] = df_defun['GRU_ED1_STR'].map(mapeos['GRU_ED1'])

print("   Campos creados: A_DEFUN_DESC, SEXO_DESC, EST_CIVIL_DESC, SIT_DEFUN_DESC, GRU_ED1_DESC")

# 2.6. Homologar causa básica de muerte (CIE-9)
print("\n6. HOMOLOGANDO CAUSA BÁSICA (CIE-9)...")
cie = pd.read_excel("data/raw/Referenciales/CIE_9_10.xls")
cie['CIE9'] = cie['CIE9'].astype(str)

# Eliminar duplicados en CIE
cie_unico = cie.drop_duplicates(subset=['CIE9'])
cie_dict = dict(zip(cie_unico['CIE9'], cie_unico['LITERAL9']))

print("   Creando diccionario CIE-9...")
print(f"   Total códigos CIE-9 únicos: {len(cie_dict):,}")
print(f"   Registros antes del merge: {len(df_defun):,}")

# Merge directo primero
print("   Realizando cruce exacto...")
df_defun = df_defun.merge(cie_unico[['CIE9', 'LITERAL9']], 
                          left_on='C_BAS1', right_on='CIE9', how='left')
df_defun.rename(columns={'LITERAL9': 'C_BAS1_DESC'}, inplace=True)
df_defun.drop('CIE9', axis=1, inplace=True, errors='ignore')

print(f"   Registros después del merge: {len(df_defun):,}")

coincidencias_exactas = df_defun['C_BAS1_DESC'].notna().sum()
print(f"   Coincidencias exactas: {coincidencias_exactas:,} ({coincidencias_exactas/len(df_defun)*100:.2f}%)")

# Búsqueda optimizada para los restantes
sin_coinc = df_defun['C_BAS1_DESC'].isna()
if sin_coinc.sum() > 0:
    print(f"   Buscando con variaciones: {sin_coinc.sum():,} registros...")
    
    # Aplicar búsqueda vectorizada
    tqdm.pandas(desc="   Procesando")
    df_defun.loc[sin_coinc, 'C_BAS1_DESC'] = df_defun.loc[sin_coinc, 'C_BAS1'].progress_apply(
        lambda x: buscar_cie9_optimizado(x, cie_dict)
    )
    
    nuevas_coinc = df_defun.loc[sin_coinc, 'C_BAS1_DESC'].notna().sum()
    print(f"   Nuevas coincidencias: {nuevas_coinc:,}")

total_coinc = df_defun['C_BAS1_DESC'].notna().sum()
print(f"   Total coincidencias: {total_coinc:,} ({total_coinc/len(df_defun)*100:.2f}%)")

# =============================================================================
# 3. RESULTADOS FINALES
# =============================================================================

print("\n" + "=" * 60)
print("RESUMEN FINAL")
print("=" * 60)
print(f"Total registros procesados: {len(df_defun):,}")
print(f"Período: 1979-1991")
print(f"Columnas totales: {len(df_defun.columns)}")
print("\nCampos principales:")
print(f"  - Ubicación defunción: {df_defun['NOMBRE_DEPARTAMENTO'].notna().sum():,}")
print(f"  - Lugar residencia: {(df_defun['NOMBRE_DEPARTAMENTO_RESIDENCIA'] != 'NO REGISTRA').sum():,}")
print(f"  - Causa básica (CIE-9): {df_defun['C_BAS1_DESC'].notna().sum():,}")
print(f"  - Campos descriptivos: 5")

print("\n✅ PROCESO COMPLETADO")
print("\nDataFrame final: df_defun")
print(f"Shape: {df_defun.shape}")

# Mostrar muestra
print("\nMuestra de datos:")
print(df_defun[['ANO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO', 
               'SEXO_DESC', 'GRU_ED1_DESC', 'C_BAS1_DESC']].head(10))

ANÁLISIS EXPLORATORIO DE DATOS - DEFUNCIONES 1979-1991

1. LEYENDO ARCHIVOS DE DEFUNCIONES...


Leyendo archivos: 100%|██████████| 13/13 [00:02<00:00,  6.15it/s]


   Total registros: 1,869,025
   Columnas: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']

2. LEYENDO DIVIPOLA...
   Columnas disponibles: ['Código_Departamento', 'Nombre_Departamento', 'Código_Municipio', 'Nombre_Municipio', 'Código_Entidad', 'Nombre_Entidad', 'Tipo', 'Longitud', 'Latitud']
   Usando columnas: Código_Municipio, Nombre_Departamento, Nombre_Municipio
   Registros únicos: 1,122
   Muestra:
   COD_DPTO COD_MUNIC NOMBRE_DEPARTAMENTO NOMBRE_MUNICIPIO
0        05       001           ANTIOQUIA         MEDELLÍN
28       05       002           ANTIOQUIA        ABEJORRAL
32       05       004           ANTIOQUIA         ABRIAQUÍ

3. HOMOLOGANDO UBICACIÓN DE DEFUNCIÓN...
   Registros iniciales: 1,869,025
   Ejemplo códigos en datos: DPTO=50, MUN=001
   Códigos únicos en DIVIPOLA: 1,122
   Registros después del merge: 1,869,025
   Cabeceras municipales: 33
   Registros despu

   Procesando: 100%|██████████| 514995/514995 [00:01<00:00, 365220.33it/s]


   Nuevas coincidencias: 449,670
   Total coincidencias: 1,803,700 (96.50%)

RESUMEN FINAL
Total registros procesados: 1,869,025
Período: 1979-1991
Columnas totales: 25

Campos principales:
  - Ubicación defunción: 1,869,025
  - Lugar residencia: 1,827,548
  - Causa básica (CIE-9): 1,803,700
  - Campos descriptivos: 5

✅ PROCESO COMPLETADO

DataFrame final: df_defun
Shape: (1869025, 25)

Muestra de datos:
    ANO NOMBRE_DEPARTAMENTO   NOMBRE_MUNICIPIO  SEXO_DESC      GRU_ED1_DESC  \
0  1979                META      VILLAVICENCIO   FEMENINO   DE 35 A 39 AÑOS   
1  1979                META           CUBARRAL   FEMENINO    DE 1 A 5 MESES   
2  1979                META           CUBARRAL   FEMENINO  DE 85 Y MÁS AÑOS   
3  1979                META  SAN JUAN DE ARAMA  MASCULINO   DE 55 A 59 AÑOS   
4  1979                META      FUENTE DE ORO  MASCULINO   DE 40 A 44 AÑOS   
5  1979                META      FUENTE DE ORO   FEMENINO   DE 6 A 11 MESES   
6  1979                META      PUERT

In [5]:
df_defun.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,...,NOMBRE_MUNICIPIO,NOMBRE_DEPARTAMENTO_RESIDENCIA,NOMBRE_MUNICIPIO_RESIDENCIA,A_DEFUN_DESC,SEXO_DESC,EST_CIVIL_DESC,SIT_DEFUN_DESC,GRU_ED1_STR,GRU_ED1_DESC,C_BAS1_DESC
0,50,001,1,1979,01,2,14,1,50,001,...,VILLAVICENCIO,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,SOLTERO,HOSPITAL O CLINICA,14,DE 35 A 39 AÑOS,ABORTO N.E.COMPLICADO POR INFECCION DEL TRACTO...
1,50,223,1,1979,01,2,04,1,50,223,...,CUBARRAL,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,SOLTERO,CASA,04,DE 1 A 5 MESES,ACIDOSIS
2,50,223,1,1979,01,2,24,1,50,223,...,CUBARRAL,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,SOLTERO,CASA,24,DE 85 Y MÁS AÑOS,ACIDOSIS
3,50,683,1,1979,01,1,18,2,50,683,...,SAN JUAN DE ARAMA,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,CASADO,CASA,18,DE 55 A 59 AÑOS,ACC.OTRA NATUR. O NATUR. NO ESPECIF. DE VHM/PE...
4,50,287,1,1979,01,1,15,1,50,287,...,FUENTE DE ORO,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,SOLTERO,CASA,15,DE 40 A 44 AÑOS,"ENVEN.DERIVADOS OXAZOLIDINA:PARAMETADIONA,TRIM..."
